# Programación Entera: Heurísticas para el Problema de la Mochila

#### Descripción

[TODO]

#### Autor
  
  * Sergio García Prado - [garciparedes.me](https://garciparedes.me)
  
#### Fecha

  * Mayo de 2018


## Contenidos
  * [Introducción](#Introducción)
  * [Problema de la Mochila](#Problema-de-la-Mochila)
    * [Mochila Simple](#Mochila-Simple)
    * [Mochila Múltiple](#Mochila-Múltiple)
    * [Mochila Multidimensional](#Mochila-Multidimensional)
  * [Problema de Empaquetamiento](#Problema-de-Empaqutamiento)
  * [Conclusiones](#Conclusiones)

## Introducción

#### Configuración del Entorno

In [1]:
import numpy as np
from ortools.linear_solver import pywraplp
from ortools.algorithms import pywrapknapsack_solver
from pathlib import Path
import re

#### Constantes

In [2]:
datasets_root = Path('./data/')
datasets_knapsack_path = datasets_root / 'knapsack'
datasets_simple_path =  datasets_knapsack_path / 'simple'
datasets_multidimensional_path =  datasets_knapsack_path / 'multidimensional'

#### Funciones de Apoyo

In [3]:
def read_simple_knapsack(file_path, value_format = float):
    l = dict()
    with open(file_path, 'r') as content_file:
        raw = np.array(list(map(value_format, content_file.read().strip().split())))
        l['n'] = raw[0].astype(int)
        l['capacity'] = raw[1]
        l['value'] = raw[np.arange(3, raw.shape[0], step=3)]
        l['weight'] = raw[np.arange(4, raw.shape[0], step=3)]
    return l

In [4]:
def read_multidimensional_knapsack(file_path, value_format = float):
    l = dict()
    with open(file_path, 'r') as content_file:
        raw = np.array(list(map(value_format, content_file.read().strip().split())))
        l['n'] = raw[0].astype(int)
        l['m'] = raw[1].astype(int)
        l['objval'] = raw[2]
        l['value'] = raw[3 + np.arange(0, l['n'])]
        l['weight'] = np.reshape(raw[3 + np.arange(l['n'], l['n'] *  (1 + l['m']))],(l['n'], l['m']))
        l['capacity'] = raw[(3 + l['n'] * (1 + l['m']) ):]
    return l

In [5]:
def pretty(d):
    for key, value in d.items():
        print(key, ':', sep='')
        print(value,'\n', sep='')

## Problema de la Mochila

#### Descripción:
TODO

### Mochila Simple

#### Descripción:
TODO

#### Modelo:
TODO

In [6]:
def knapsack_simple_exact(p):
    solver = pywrapknapsack_solver.KnapsackSolver(
        pywrapknapsack_solver.KnapsackSolver.
        KNAPSACK_MULTIDIMENSION_BRANCH_AND_BOUND_SOLVER,
        'test')

    solver.Init(p['value'], [p['weight'].tolist()], 
                [p['capacity'].tolist()])
    solution = solver.Solve()

    r = dict()
    if solution:
        r['objval'] = np.array(solution)
        r['solution'] = np.array([1 if solver.BestSolutionContains(x) else 0
                                  for x in range(0, len(p['weight'].tolist()))])
    else:
        raise ValueError('No solution found.')
    return r

In [7]:
gen100_1 = read_simple_knapsack(datasets_simple_path / 'gen100_1.txt')

In [8]:
pretty(knapsack_simple_exact(gen100_1))

objval:
1753

solution:
[1 0 1 1 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 1
 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0 0 1 0 1 1 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0
 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0]



#### Heurísticas:
TODO

### Ejemplos

##### gen100.1

##### gen100.2

##### gen100.3

##### gen100.4

##### gen100.5

##### gen200.1

##### gen200.2

##### gen200.3

##### gen200.4

##### gen200.5

##### gen300.1

##### gen300.2

##### gen300.3

##### gen300.4

##### gen300.5

### Mochila Múltiple

#### Descripción:
TODO

#### Modelo:
TODO

In [9]:
def knapsack_multiple_exact(p, m = 1):
    solver = pywraplp.Solver('SolveSimpleSystem',
                           pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
    
    objective = solver.Objective()
    
    x = list()
    for i in range(m):
        x.append(list())
        for j in range(int(p['n'])):
            x[i].append(solver.IntVar(0, 1, "x(" + str(i) + ", " + str(j) + ")"))
            objective.SetCoefficient(x[i][j], float(p['value'][j]))
    
    objective.SetMaximization()
    
    constraints = list()
    for i in range(p['n']):
        constraints.append(solver.Constraint(0, 1))
        for j in range(m):
            constraints[i].SetCoefficient(x[j][i], 1)
    
    for j in range(m):
        constraints.append(solver.Constraint(0, p['capacity']))
        for i in range(p['n']):
            constraints[j + p['n']].SetCoefficient(x[j][i], p['weight'][i])
            pass
    status = solver.Solve()
    
    if status == solver.OPTIMAL or status == solver.FEASIBLE:
        r = dict()
        r['objval'] = 0
        r['solution'] = np.zeros((m, p['n']), dtype=int)
        for i in range(m):
            for j in range(p['n']):
                r['solution'][i][j] = x[i][j].solution_value()
                r['objval'] += x[i][j].solution_value() * p['value'][j]
        
        return r
    else: 
        raise ValueError('The solver could not solve the problem.')

#### Heurísticas:
TODO

### Ejemplos

In [10]:
gen10_1 = read_simple_knapsack(datasets_simple_path / 'gen10_1.txt')

In [11]:
pretty(knapsack_multiple_exact(gen10_1, 3))

objval:
2286.0

solution:
[[0 1 0 0 0 0 1 0 1 0 0 1 0 1 0 0 1 0 1 1 0 1 0 0 1 0 0 1 0 1 0 1 0 0 0 1
  0 1 0 0 1 0 0 1 1 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0
  0 0 0 1 0 1 0 0 0 0 0 0 1 0]
 [1 0 1 1 1 1 0 1 0 1 1 0 0 0 1 1 0 1 0 0 1 0 1 0 0 1 1 0 1 0 1 0 1 0 0 0
  1 0 1 0 0 0 1 0 0 0 0 1 0 1]]



##### gen100.1

In [12]:
pretty(knapsack_multiple_exact(gen100_1, 1))

objval:
1753.0

solution:
[[1 0 1 1 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0
  1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0 0 1 0 1 1 0 0 0 1 0 0 1 0 0 0 1 0 0
  0 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0]]



##### gen100.2

##### gen100.3

##### gen100.4

##### gen100.5

##### gen200.1

##### gen200.2

##### gen200.3

##### gen200.4

##### gen200.5

##### gen300.1

##### gen300.2

##### gen300.3

##### gen300.4

##### gen300.5

### Mochila Multidimensional

#### Descripción:
TODO

#### Modelo:
TODO

In [13]:
def knapsack_multidimensional_exact(p):
    solver = pywrapknapsack_solver.KnapsackSolver(
        pywrapknapsack_solver.KnapsackSolver.
        KNAPSACK_MULTIDIMENSION_BRANCH_AND_BOUND_SOLVER,
        'test')

    solver.Init(p['value'].tolist(), p['weight'].tolist(), 
                p['capacity'].tolist())
    solution = solver.Solve()

    r = dict()
    if solution:
        r['objval'] = np.array(solution)
        r['solution'] = np.array([1 if solver.BestSolutionContains(x) else 0
                                  for x in range(0, len(p['weight'].tolist()))])
    else:
        raise ValueError('No solution found.')
    return r

#### Heurísticas:
TODO

### Ejemplos

##### mknap1.2

In [14]:
mknap1_2 = read_multidimensional_knapsack(datasets_multidimensional_path / 'mknap1_2.dat')

In [15]:
pretty(mknap1_2)

n:
10

m:
10

objval:
8706.1

value:
[ 600.1  310.5 1800.  3850.    18.6  198.7  882.  4200.   402.5  327. ]

weight:
[[ 20.   5. 100. 200.   2.   4.  60. 150.  80.  40.]
 [ 20.   7. 130. 280.   2.   8. 110. 210. 100.  40.]
 [ 60.   3.  50. 100.   4.   2.  20.  40.   6.  12.]
 [ 60.   8.  70. 200.   4.   6.  40.  70.  16.  20.]
 [ 60.  13.  70. 250.   4.  10.  60.  90.  20.  24.]
 [ 60.  13.  70. 280.   4.  10.  70. 105.  22.  28.]
 [  5.   2.  20. 100.   2.   5.  10.  60.   0.   0.]
 [ 45.  14.  80. 180.   6.  10.  40. 100.  20.   0.]
 [ 55.  14.  80. 200.   6.  10.  50. 140.  30.  40.]
 [ 65.  14.  80. 220.   6.  10.  50. 180.  30.  50.]]

capacity:
[450. 540. 200. 360. 440. 480. 200. 360. 440. 480.]



In [16]:
pretty(knapsack_multidimensional_exact(mknap1_2))

objval:
8705

solution:
[0 1 0 1 1 0 0 1 0 1]



##### mknap1.3

##### mknap1.4

##### mknap1.6

##### mknap1.7

##### mknapcb1.1

##### mknapcb1.2

##### mknapcb1.3

##### mknapcb1.4

##### mknapcb1.5

## Conclusiones